In [1]:
# Добавляем директорию для равномерного импорта всего с корневой локации
import sys

sys.path.append('../')

# Импортируем библиотеку, которая автоматически настраивает коннектор для системы выполнения кода
from src.libs.LCTWrapTwin import start_module

In [ ]:
# ЭТОТ БЛОК КОДА МОЖНО И НУЖНО МОДИФИЦИРОВАТЬ
# Как обычно, подсказки - в импортах

from typing import Literal
from src.libs.LCTWrapTwin.Modules.Handler import MissionHandler, TrustedHandler
import socket
import threading
import time

class UserMissionHandler(MissionHandler):

    @staticmethod
    def config_cyber_obstacles() -> dict:

        return {
            "CybP_01": True,  # постоянно (автопилот) - хэш автопилота
            "CybP_02": True,  # постоянно (неполадки связи, смешное сообщение)
            "CybP_03": True,  # постоянно (привода, поддел инфы о них) - хэш приводов
            "CybP_04": True,  # постоянно (ускорение щетки)
            "CybP_05": True,
            "CybP_06": True,  # постоянно (несанкц. ускорение)
        }
    

    def change_barrier_UDP(self, do: Literal["up"] | Literal["drop"]) -> None:  # Смена положения шлагбаума путем перехвата трафика [UDP] #

        def periodic_ap_barrier():
            future_result = 1 if do == "up" else 0
            while True:
                self.set_barrier_toggle()
                if UserTrustedHandler.check_CybP_05_UDP(self, future_result):
                    break

        ap_barrier = threading.Thread(target=periodic_ap_barrier, daemon=True)
        ap_barrier.start()

    def intercept_UDP(self, name: str) -> int:  # Перехват трафика умных устройств [UDP] #

        try:
            ports = {
            "traffic_0": 5030,  # светофор 0 (самый ближний)
            "traffic_1": 5031,  # светофр 1 (средний)
            "traffic_2": 5032,  # светофор 2 (самый дальний)
            "barrier": 15000,   # шлагбаум
            "autobot": 10004    # автоматичский робот
            }

        except Exception as e:
            self.lg.log(f"Переданный в параметр name аргумент: {name}!")

        listen_port = ports[name]
        result = None

        def periodic_interceptor():
            nonlocal result
            
            interceptor_socket = None
            forward_socket = None
            
            try:
                interceptor_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
                interceptor_socket.settimeout(0.3)
                interceptor_socket.bind(('127.0.0.1', listen_port))
                
                forward_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
                
                try:
                    data, client_address = interceptor_socket.recvfrom(1024)
                    decoded_data = data.decode('utf-8', errors='ignore').strip()
                    if name in tuple(ports.keys())[:-2]:  # светофоры
                        color = int(decoded_data[-2])
                        self.lg.log(f"{color}")
                        self.lg.log(f"{name} КРАСНЫЙ") if color == 1 else self.lg.log(f"{name} ЗЕЛЁНЫЙ")
                        if color == 2:
                            result = True
                    elif name in tuple(ports.keys())[-2]:  # шлагбаум
                        status = decoded_data[-2]
                        result = int(status)
                    elif name in tuple(ports.keys())[-1]:  # автоматичский робот
                        result = decoded_data

                except socket.timeout:
                    self.lg.log("Отключение от порта, ДАННЫЕ НЕ ПОЛУЧЕНЫ!!!")
                    
                except Exception as e:
                    self.lg.log(f"Ошибка при получении данных: {e}")
                    
            except Exception as e:
                self.lg.log(f"Ошибка создания сокета: {e}")      
          
            finally:
                try:
                    if interceptor_socket:
                        interceptor_socket.close()
                    if forward_socket:
                        forward_socket.close()
                except:
                    pass
        
        interceptor_thread = threading.Thread(target=periodic_interceptor, daemon=True)
        interceptor_thread.start()
        interceptor_thread.join(timeout=0.25)
        return result

    def check_autobot_UDP(self) -> bool:  # Проверка положения автобота перехватом трафика [UDP] #
        
        def check_distance() -> bool:  # првоеряет факт увеличения расстояния между роботами
            flag = False
            while True:
                for m in self.get_message_from_trusted_module():
                    if m["type"] == "my_coords":
                        my_x, my_y = m["coords"]["position_x"], m["coords"]["position_y"]
                        flag = True
                        break
                if flag:
                    break
            
            check = []
            for _ in range(2):        
                data_autorobot = self.intercept_UDP("autobot")[1:-1].split(',')
                coords = []
                for i in range(2):
                    coord = float(data_autorobot[i].split(':')[1])
                    coords.append(coord)
                    if i == 1:
                        time.sleep(1)
                autorobor_x, autorobor_y = coords[0], coords[1]
                d = round(sqrt((autorobor_x - my_x)**2 + (autorobor_y - my_y)**2), 4)
                check.append(d)

            # self.lg.log(f"{check}")
            return check[0] < check[1]  # возвращает True, если расстоние между роботами уменьшается


        flag_stop = False
        while True:  # если робот стоит на месте; если двигается -> запускаем другой цикл и ждем остановки
            new_1 = self.intercept_UDP("autobot")
            time.sleep(1.0)
            new_2 = self.intercept_UDP("autobot")
            if new_1 == new_2:
                if not flag_stop:
                    flag_stop = True
                    self.lg.log("[📡<OP>🛜]::💤 Пока что авторобот стоит на месте. Ожидаем... 🛑")
                continue
            else:
                self.lg.log("[📡<OP>🛜]::🌀 Авторобот находится в движении. Ожидаем... 🛑")
                break
        
        check = False   
        while True:
            new_1 = self.intercept_UDP("autobot")
            time.sleep(1.0)
            new_2 = self.intercept_UDP("autobot")
            if new_1 == new_2:
                self.lg.log("[📡<OP>🛜]::💤 Авторобот остановился, можно ехать ♻️")
                break
            elif not check:
                result_d = [check_distance() for _ in range(2)]
                if all(result_d):
                    self.lg.log("[📡<OP>🛜]::🌀 Авторобот находится в движении - он уезжает от нас! Вперед... 🟢")
                    break
                check = True

        return True

    def check_t_light_CV(self) -> int:  # Проверка светофора на зеленый/красный [КЗ] #

        velocity = 20
        green = False
        red = False
        self.lg.log("🚦 Внимание! Светофор 🚦")

        while True:
            frame = self.get_camera_frame()
            center_row = frame[240, :]
            
            green_pixels = 0
            red_pixels = 0
            
            for x in range(len(center_row)):
                pixel = center_row[x]
                b, g, r = pixel[0].astype(int), pixel[1].astype(int), pixel[2].astype(int)
                
                is_green = (b in range(40-velocity, 40+velocity)) and (g in range(200-velocity, 200+velocity)) and (r in range(40-velocity, 40+velocity))
                
                is_red = (b in range(40-velocity, 40+velocity)) and (g in range(40-velocity, 40+velocity)) and (r in range(200-velocity, 200+velocity))
                
                if is_green:
                    green_pixels += 1      
                elif is_red:
                    red_pixels += 1
                
            if green_pixels > red_pixels:
                if not green:
                    green = True
                    self.lg.log("🟢 Светофор ЗЕЛЁНЫЙ. Вперед! 🟢")
                break
            else:
                if not red:
                    red = True
                    self.lg.log("🔴 Светофор КРАСНЫЙ. Ожидаем... 🔴")
                time.sleep(1.0)

        return True 

    def check_pedestrian_CV(self) -> bool:  # Проверка положения пешехода [КЗ] #
        BGR_TARGET_1 = (65, 110, 200)  # приблизительные bgr для конуса без тени
        BGR_TARGET_2 = (80, 80, 80)  # приблизительные bgr для конуса в тени
        x, y = 323, 426
        velocity = 55
        wait = False
        
        while True:
            frame = self.get_camera_frame()
            pixels = frame[y, x-2:x+3]
            
            mas_1 = [[pix[i] in range(BGR_TARGET_1[i] - velocity, BGR_TARGET_1[i] + velocity) for i in range(3)] for pix in pixels]
            mas_2 = [[pix[i] in range(BGR_TARGET_2[i] - velocity, BGR_TARGET_2[i] + velocity) for i in range(3)] for pix in pixels]
            
            result_1 = any([all(i) for i in mas_1])
            result_2 = any([all(i) for i in mas_2])
            
            if result_1 or result_2:
                self.lg.log(f"🚶‍♂️Пешеход перешел дорогу. Вперед!🚶‍♂️")
                break
            else:
                if not wait:
                    wait = True
                    self.lg.log(f"🚸 Пешеходный переход. Ожидаем... 🚸")
                time.sleep(0.2)
                continue
        return True

    def change_barrier_CV(self) -> int:  # Смена положения шлагбаума [КЗ] #
        Flag = True
        while Flag:
            buffer = self.get_message_from_trusted_module()
            if len(buffer):
                for message in buffer:
                    if "type" in message.keys():
                        if message["type"] == "status_barrier":
                            start_status =  message["status"]
                            Flag = False
                            break
                if Flag:
                    # self.lg.log("Нет необходимых сообщений!")
                    time.sleep(0.2)
            else:
                # self.lg.log("Буффер пустой!")
                time.sleep(0.2)

        while True:
            self.set_barrier_toggle()
            time.sleep(1.7)  # ждем ответ от МБ (процесс поднятия + время получения ответа)
            current_status = self.get_message_from_trusted_module()[-1]["status"]
            if start_status != current_status:
                break
   
    def mission_code(self):
       
        def _handle_speed_control(message):
            action = message["action"]
            max_speed = message["max_speed"]
            reason = message["reason"]
            
            self.lg.log(f"🎛️ АВТОПИЛОТ: Получена команда управления скоростью")
            self.lg.log(f"   Действие: {action}")
            self.lg.log(f"   Максимальная скорость: {max_speed}")
            self.lg.log(f"   Причина: {reason}")
            
            if action == "emergency_slow":
                self.lg.warn("⚠️ АВТОПИЛОТ: Экстренное снижение скорости!")
                self.set_robot_speed(max_speed)

                
            else:
                self.lg.warn(f"⚠️ АВТОПИЛОТ: Неизвестное действие: {action}")
                
        def _process_trusted_message(message):
            try:
                msg_type = message["type"]
                
                if msg_type == "speed_control":
                    _handle_speed_control(message)
                elif msg_type == "my_coords":
                    pass # Работает 
                elif msg_type == "status_barrier":
                    pass # Работает 
                
            except Exception as e:
                self.lg.error(f"❌ АВТОПИЛОТ: Ошибка обработки сообщения: {e}")
        
        
        def _buffer_message_handler():
            while True:
                try:
                    messages = self.get_message_from_trusted_module()
                    
                    if len(messages) != 0: 
                        for message in messages:
                            _process_trusted_message(message)
                    
                    time.sleep(0.5)  
                    
                except Exception as e:
                    self.lg.error(f"❌ АВТОПИЛОТ: Ошибка в обработке сообщений: {e}")
                    time.sleep(0.1)
                    

        message_thread = threading.Thread(target=_buffer_message_handler, daemon=True)
        message_thread.start()



        def robot_marshrut():
            self.lg.log("Сообщение обычное")
            self.lg.warn("Сообщение о предупреждении")
            self.lg.error("Сообщение об ошибке")

            self.lg.log("Тестовое сообщение блока кода для выполнения заезда")
            self.set_robot_speed(0.24)
            self.set_brush_speed(100)
            self.change_barrier_UDP("drop")  # Сразу закрываем шлагбаум дистанционно с помощью пакетов
            self.ap_hook.do_rotate({"x": 0.6, "y": 1})
            self.ap_hook.do_move({"x": 0.6, "y": 1})
            self.ap_hook.do_rotate({"x": 0.68, "y": 1.4})
            self.ap_hook.do_move({"x": 0.68, "y": 1.4})  # Шлагбаум 
            self.change_barrier_CV()

            self.ap_hook.do_move({"x": 0.68, "y": 2.35})  # Пешеход верхний
            self.check_pedestrian_CV()
            self.change_barrier_UDP("drop")  # Снова закрываем шлагбаум дистанционно с помощью пакетов

            self.ap_hook.do_move({"x": 0.68, "y": 2.92})
            self.ap_hook.do_rotate({"x": 1, "y":2.92})
            self.ap_hook.do_move({"x": 1, "y": 2.92})  # Возможное столкновение с роботом (первый чек)
            self.check_autobot_UDP()
            
            self.ap_hook.do_move({"x": 2.28, "y":2.92})
            self.ap_hook.do_rotate({"x": 2.28, "y":3.32})
            self.ap_hook.do_move({"x": 2.28, "y":3.32})
            self.ap_hook.do_rotate({"x": 2.6, "y":3.32})
            self.ap_hook.do_move({"x": 2.7, "y":3.32})
            self.ap_hook.do_rotate({"x": 2.6, "y":3.4})
            self.ap_hook.do_move({"x": 2.6, "y":3.4})
            self.ap_hook.do_move({"x": 2.16, "y":3.4})
            self.ap_hook.do_rotate({"x": 2.12, "y":3.08})
            self.ap_hook.do_move({"x": 2.12, "y":3.08})
            self.ap_hook.do_rotate({"x": 1.8, "y":3.08})
            self.ap_hook.do_move({"x": 1.32, "y":3.08})
            self.ap_hook.do_rotate({"x": 1.32, "y":2.12})
            self.ap_hook.do_move({"x": 1.32, "y":2.12})
            self.ap_hook.do_rotate({"x": 1.8, "y":2.12})
            self.ap_hook.do_move({"x": 1.8, "y":2.12})
            self.ap_hook.do_move({"x": 2.6, "y":2.12})
            self.ap_hook.do_rotate({"x": 2.65, "y":2.2})
            self.ap_hook.do_move({"x": 2.65, "y":2.2})
            self.ap_hook.do_rotate({"x": 2.6, "y":2.28})
            self.ap_hook.do_move({"x": 2.6, "y":2.28})
            self.ap_hook.do_rotate({"x": 1.2, "y":2.28})
            self.ap_hook.do_move({"x": 1.8, "y":2.28})  # Светофор 1 (центральный)
            self.check_t_light_CV()

            self.ap_hook.do_move({"x": 1.36, "y":2.28})
            self.ap_hook.do_rotate({"x": 1.32, "y":0.8})
            self.ap_hook.do_move({"x": 1.32, "y":1})  # Светофор 0 (нижний)
            self.check_t_light_CV()

            self.ap_hook.do_move({"x": 1.32, "y":0.8})
            self.ap_hook.do_rotate({"x": 1.4, "y":0.6})
            self.ap_hook.do_move({"x": 1.4, "y":0.6})
            self.ap_hook.do_rotate({"x": 1.48, "y":1})
            self.ap_hook.do_move({"x": 1.48, "y":1})
            self.ap_hook.do_move({"x": 1.48, "y":1.4})
            self.ap_hook.do_move({"x": 1.48, "y":1.8})
            self.ap_hook.do_move({"x": 1.48, "y":2.2})
            self.ap_hook.do_move({"x": 1.48, "y":2.6})  # Возможное столкновение с роботом (второй чек)
            self.check_autobot_UDP()
 
            self.ap_hook.do_move({"x": 1.48, "y":3.08})
            self.ap_hook.do_rotate({"x": 0.52, "y":3.08})
            self.ap_hook.do_move({"x": 0.52, "y":3.08})
            self.ap_hook.do_rotate({"x": 0.52, "y":3.08})
            self.ap_hook.do_move({"x": 0.52, "y":3.08})
            self.ap_hook.do_rotate({"x": 0.52, "y":2.85})
            self.ap_hook.do_move({"x": 0.52, "y":2.85})  # Верхний пешехов        
            self.check_pedestrian_CV()

            self.ap_hook.do_move({"x": 0.52, "y":2.2})  # Шлагбаум 
            self.change_barrier_CV()

            self.ap_hook.do_move({"x": 0.52, "y":1.8})
            self.ap_hook.do_move({"x": 0.52, "y":1.4})
            self.ap_hook.do_move({"x": 0.52, "y":1})
            self.ap_hook.do_rotate({"x": 0.52, "y":0.52})
            self.ap_hook.do_move({"x": 0.52, "y":0.52})
            self.ap_hook.do_rotate({"x": 1, "y":0.52})
            self.ap_hook.do_move({"x": 1, "y":0.52})

            self.ap_hook.do_move({"x": 1.4, "y":0.52})
            self.ap_hook.do_move({"x": 1.8, "y":0.52})
            self.ap_hook.do_move({"x": 1.8, "y":0.52}) 
            self.ap_hook.do_move({"x": 1.9, "y":0.52})
            self.ap_hook.do_move({"x": 1.95, "y":0.52})  # Пешеход нижний 
            self.check_pedestrian_CV()

            self.ap_hook.do_move({"x": 2.2, "y":0.52})
            self.ap_hook.do_move({"x": 2.6, "y":0.52})    
            self.ap_hook.do_rotate({"x": 3.08, "y":0.52})
            self.ap_hook.do_move({"x": 3.08, "y":0.52})
            self.ap_hook.do_rotate({"x": 3.08, "y":1})  # Возможное столкновение с роботом (третий чек)
            self.check_autobot_UDP()

            self.ap_hook.do_move({"x": 3.08, "y":1})
            self.ap_hook.do_move({"x": 3.08, "y":1.4})
            self.ap_hook.do_move({"x": 3.08, "y":1.8})
            self.ap_hook.do_move({"x": 3.08, "y":2.2})
            self.ap_hook.do_move({"x": 3.08, "y":2.6})
            self.ap_hook.do_move({"x": 3.08, "y":3.0})
            # self.check_autobot_UDP() !!! НЕ ЗДЕСЬ !!!
            self.ap_hook.do_rotate({"x": 3, "y":3.4})
            self.ap_hook.do_move({"x": 3.0, "y":3.4})
            self.ap_hook.do_rotate({"x": 2.92, "y":3})
            self.ap_hook.do_move({"x": 2.92, "y":3})
            self.ap_hook.do_rotate({"x": 2.92, "y":1.0})
            self.ap_hook.do_move({"x": 2.92, "y":1.0})
            self.ap_hook.do_move({"x": 2.92, "y":0.68})
            self.ap_hook.do_rotate({"x": 2.6, "y":0.68})
            self.ap_hook.do_move({"x": 2.6, "y":0.68})
            self.ap_hook.do_move({"x": 2.45, "y":0.68})  # Пешеход нижний
            self.check_pedestrian_CV()

            self.ap_hook.do_move({"x": 1.8, "y":0.68})
            self.ap_hook.do_move({"x": 1.4, "y":0.68}) 
            self.ap_hook.do_move({"x": 1.0, "y":0.68})
            self.ap_hook.do_rotate({"x": 0.68, "y":0.68})
            self.ap_hook.do_move({"x": 0.68, "y":0.68})
            self.ap_hook.do_rotate({"x": 0.6, "y":1.0})
            self.ap_hook.do_move({"x": 0.6, "y":1.0})
            self.ap_hook.do_rotate({"x": 0.2, "y":1.0})
            self.ap_hook.do_move({"x": 0.2, "y":1.0})
            self.ap_hook.do_rotate({"x": 0.6, "y":1.0}) 

        robot_marshrut()



class UserTrustedHandler(TrustedHandler):
    def __init__(self, context):
        super().__init__(context)

        # Переменные для защиты от CybP_01 (компрометация автопилота)
        self.cybp01_attack_detected = False
        self.cybp01_attack_count = 0
        self.cybp01_original_hash = None
        self.cybp01_current_hash = None
        self.cybp01_monitoring_active = False
        self.cybp01_position_monitoring = False
                
        # Переменные для защиты от CybP_02 (компрометация связи)
        self.cybp02_message_count = 0
        self.cybp02_protection_active = False
        self.cybp02_malfunction_detected = False
        self.cybp02_attack_count = 0
        self.initial_message = "initial_secure_message"
        self.last_correct_message = None
                
        # Переменные для защиты от CybP_03 (компрометация приводов)
        self.cybp03_attack_detected = False
        self.cybp03_attack_count = 0
        self.cybp03_compromised_drive_id = None
        self.cybp03_last_drive_data = None
        self.cybp03_monitoring_active = False

        # Переменные для защиты от CybP_04 (компрометация оборудования)
        self.brush_speed_threshold = 150 
        self.normal_brush_speed = 100    
        self.cybp04_attack_detected = False
        self.cybp04_attack_count = 0
        self.last_brush_speed = 0
        
        # Переменные для защиты от CybP_05 (компрометация шлагбаума)
        self.check_barrier = False
        self.cybp_05_udp = False

        # Переменные для защиты от CybP_06 (нарушение скорости)
        self.cybp06_attack_detected = False
        self.cybp06_attack_count = 0
        self.cybp06_current_cell = None
        self.cybp06_speed_threshold = 0.35 
        self.cybp06_normal_speed = 0.24      
        self.cybp06_last_position = {"x": 0.2, "y": 1.0, "time": 0}
        self.cybp06_current_speed = 0

        self.threat_levels = {
            "CybP_01": "🔴 ВЫСОКИЙ",
            "CybP_02": "🟡 СРЕДНИЙ", 
            "CybP_03": "🟡 СРЕДНИЙ",
            "CybP_04": "🟢 НИЗКИЙ",
            "CybP_05": "🟢 НИЗКИЙ",
            "CybP_06": "🟡 СРЕДНИЙ"
        }


    def trusted_code(self):
        
        self.lg.log("=" * 80)
        self.lg.log("🛡️ ИНИЦИАЛИЗАЦИЯ МОДУЛЯ БЕЗОПАСНОСТИ")
        self.lg.log("=" * 80)
        
        self.lg.log("")
        self.lg.log("🚁 CybP_01: КОМПРОМЕТАЦИЯ АВТОПИЛОТА")
        self.lg.log(f"   {self.threat_levels['CybP_01']} УРОВЕНЬ УГРОЗЫ")
        self.lg.log("   🛡️ Мониторинг целостности кода автопилота")
        self.lg.log("   ✅ Система защиты готова")
        
        self.lg.log("")
        self.lg.log("🔐 CybP_02: ПОДМЕНА СООБЩЕНИЙ")
        self.lg.log(f"   {self.threat_levels['CybP_02']} УРОВЕНЬ УГРОЗЫ")
        self.lg.log(f"   📊 Начальное сообщение: '{self.initial_message}'")
        self.lg.log("   🛡️ Защита от подмены сообщений через CRC8")
        self.lg.log("   ✅ Система защиты готова")
        
        self.lg.log("")
        self.lg.log("🔧 CybP_03: КОМПРОМЕТАЦИЯ ПРИВОДОВ")
        self.lg.log(f"   {self.threat_levels['CybP_03']} УРОВЕНЬ УГРОЗЫ")
        self.lg.log("   🛡️ Мониторинг целостности данных приводов")
        self.lg.log("   ✅ Система защиты готова")
        
        self.lg.log("")
        self.lg.log("🧹 CybP_04: СБОЙ НАВЕСНОГО ОБОРУДОВАНИЯ")
        self.lg.log(f"   {self.threat_levels['CybP_04']} УРОВЕНЬ УГРОЗЫ")
        self.lg.log(f"   🔧 Порог скорости: {self.brush_speed_threshold}, норма: {self.normal_brush_speed}")
        self.lg.log("   🛡️ Мониторинг скорости щетки")
        self.lg.log("   ✅ Система защиты готова")
        
        self.lg.log("")
        self.lg.log("🚗 CybP_05: КОМПРОМЕТАЦИЯ ШЛАГБАУМА")
        self.lg.log(f"   {self.threat_levels['CybP_05']} УРОВЕНЬ УГРОЗЫ")
        self.lg.log("   🛡️ Распознавание шлагбаума включено")
        self.lg.log("   ✅ Система защиты готова")
        
        self.lg.log("")
        self.lg.log("🚗 CybP_06: НЕСАНКЦИОНИРОВАННОЕ УСКОРЕНИЕ")
        self.lg.log(f"   {self.threat_levels['CybP_06']} УРОВЕНЬ УГРОЗЫ")

        self.lg.log(f"   ⚡ Порог скорости: {self.cybp06_speed_threshold}, норма: {self.cybp06_normal_speed}")
        self.lg.log("   🛡️ Мониторинг скорости и позиции")
        self.lg.log("   ✅ Система защиты готова")
        
        self.lg.log("")
        self.lg.log("=" * 80)
        self.lg.log("🚀 ВСЕ СИСТЕМЫ ЗАЩИТЫ АКТИВИРОВАНЫ")
        self.lg.log("=" * 80)
        
        
        self._start_all_monitoring_systems()
        
        
        while True:
            time.sleep(1)
            # Основной цикл Модуля Безопасности
            pass

    @classmethod
    def get_last_instance(cls):
        return cls._last_instance

    def make_next_short_message(self, prev_message: str):
        
        self.cybp02_message_count += 1
        self.cybp02_protection_active = True
        
        self._detect_cybp02_attack(prev_message)
        
        return self.last_correct_message




# Атака CybP_01
    def _start_autopilot_monitoring(self):
        
        def autopilot_monitor():
            self.lg.log("Запуск мониторинга автопилота для защиты от \"CybP_01\"")
            
            self._initialize_original_hash()
            
            while True:
                try:
                    self._check_autopilot_integrity()
                    time.sleep(0.5)
                except Exception as e:
                    self.lg.error(f"❌ \"CybP_01\": Ошибка в мониторинге автопилота: {e}")
                    time.sleep(1)
        
        monitor_thread = threading.Thread(target=autopilot_monitor, daemon=True)
        monitor_thread.start()

    def _initialize_original_hash(self):
        try:
            self.lg.log("\"CybP_01\": Инициализация оригинального хеша автопилота")
            hash_data = self.get_ap_code_hash()
            if hash_data and 'ap_code_hash' in hash_data:
                self.cybp01_original_hash = hash_data['ap_code_hash']
                self.cybp01_current_hash = hash_data['ap_code_hash']
                self.lg.log(f"✅ \"CybP_01\": Оригинальный хеш автопилота сохранен: {self.cybp01_original_hash[:10]}...")
            else:
                self.lg.error("❌ \"CybP_01\": Не удалось получить оригинальный хеш автопилота")
        except Exception as e:
            self.lg.error(f"❌ \"CybP_01\": Ошибка при инициализации хеша: {e}")

    def _check_autopilot_integrity(self):
        try:
            hash_data = self.get_ap_code_hash()
            if not hash_data or 'ap_code_hash' not in hash_data:
                return
            
            current_hash = hash_data['ap_code_hash']
            self.cybp01_current_hash = current_hash
            
            if self.cybp01_original_hash and current_hash != self.cybp01_original_hash:
                if not self.cybp01_attack_detected:
                    self.cybp01_attack_detected = True
                    self.cybp01_attack_count += 1
                    
                    self.lg.warn("🔴 ОБНАРУЖЕНА КОМПРОМЕТАЦИЯ АВТОПИЛОТА CybP_01!")
                    self.lg.warn(f"   {self.threat_levels['CybP_01']} УРОВЕНЬ УГРОЗЫ")
                    self.lg.warn(f"   Оригинальный хеш: {self.cybp01_original_hash[:10]}...")
                    self.lg.warn(f"   Текущий хеш: {current_hash[:10]}...")
                    self.lg.warn(f"   Количество атак: {self.cybp01_attack_count}")
                    
                    self._apply_cybp01_protection()
                    
        except Exception as e:
            self.lg.error(f"❌ \"CybP_01\": Ошибка при проверке целостности автопилота: {e}")

    def _apply_cybp01_protection(self):
        self.lg.log("✅ ПРИМЕНЕНИЕ ЗАЩИТЫ ОТ CybP_01")
        
        try:
            self.lg.log("🔧 \"CybP_01\": Сброс автопилота для восстановления")
            reset_result = self.set_ap_force_reset()
            time.sleep(1.0)  
            
            if reset_result and isinstance(reset_result, dict) and reset_result.get('status') == 'OK':
                self.lg.log("✅ \"CybP_01\": Автопилот сброшен")
            else:
                self.lg.warn("⚠️ \"CybP_01\": Сброс не подтвержден, но продолжаем")
            
            self.lg.log(" \"CybP_01\": Ожидание восстановления автопилота")
            self._wait_for_autopilot_recovery()
            
        except Exception as e:
            self.lg.error(f"❌ \"CybP_01\": Ошибка защиты: {e}")
            self._apply_emergency_stop_cybp01()

    def _wait_for_autopilot_recovery(self):
        self.lg.log(" \"CybP_01\": Ожидание восстановления автопилота")
        
        for attempt in range(10):
            time.sleep(1.0)
            
            try:
                hash_data = self.get_ap_code_hash()
                if hash_data and 'ap_code_hash' in hash_data:
                    current_hash = hash_data['ap_code_hash']
                    if current_hash == self.cybp01_original_hash:
                        self.lg.log("✅ CybP_01: Автопилот восстановлен!")
                        self._handle_cybp01_recovery()
                        return
            except Exception as e:
                self.lg.error(f"❌ CybP_01: Ошибка при проверке: {e}")
        
        self.lg.log(" \"CybP_01\": Атака завершилась, система автоматически восстановится")
        self._handle_automatic_recovery_cybp01()

    def _handle_cybp01_recovery(self):
        self.lg.log("🔄 \"CybP_01\": ВОССТАНОВЛЕНИЕ ПОСЛЕ АТАКИ")
        self.lg.log(" \"CybP_01\": Автопилот восстановлен до оригинального состояния")
        self._reset_cybp01_state()
        self.lg.log("✅ \"CybP_01\": Система восстановлена, мониторинг продолжается")

    def _handle_automatic_recovery_cybp01(self):
        self.lg.log("🔄 \"CybP_01\": АВТОМАТИЧЕСКОЕ ВОССТАНОВЛЕНИЕ")
        self.lg.log("\"CybP_01\": Атака завершилась по таймауту, система восстановлена")
        self._reset_cybp01_state()
        self.lg.log("✅ \"CybP_01\": Система восстановлена, мониторинг продолжается")

    def _apply_emergency_stop_cybp01(self):
            self.lg.warn("🔴 \"CybP_01\": КРИТИЧЕСКАЯ ОШИБКА - ЭКСТРЕННАЯ ОСТАНОВКА")
            
            try:
                emergency_result = self.set_emergency_stop(True)
                if emergency_result and isinstance(emergency_result, dict) and emergency_result.get('status') == 'OK':
                    self.lg.log("✅ \"CybP_01\": Все приводы экстренно остановлены")
                    self._release_emergency_stop_cybp01()
                else:
                    self.lg.warn("⚠️ \"CybP_01\": Не удалось применить экстренную остановку")
            except Exception as e:
                self.lg.error(f"❌ \"CybP_01\": Ошибка экстренной остановки: {e}")

    def _reset_cybp01_state(self):
        self.cybp01_attack_detected = False
        self.cybp01_position_monitoring = False

    def _release_emergency_stop_cybp01(self):
        self.lg.log("🟢 \"CybP_01\": СНЯТИЕ ЭКСТРЕННОЙ ОСТАНОВКИ")
        
        try:
            self.lg.log("\"CybP_01\": Проверка восстановления системы...")
            
            time.sleep(1.0)
            
            release_result = self.set_emergency_stop(False)
            
            if release_result and isinstance(release_result, dict) and release_result.get('status') == 'OK':
                self.lg.log("✅ \"CybP_01\": Экстренная остановка успешно снята")
                self.lg.log(" \"CybP_01\": Робот готов к продолжению движения")
                
                self._reset_cybp01_state()
                
            else:
                self.lg.warn("⚠️ \"CybP_01\": Не удалось снять экстренную остановку")
                self.lg.warn("🔧 \"CybP_01\": Требуется ручное вмешательство")
                
        except Exception as e:
            self.lg.error(f"❌ \"CybP_01\": Ошибка при снятии экстренной остановки: {e}")
            self.lg.warn("🔧 \"CybP_01\": Требуется ручное вмешательство")


# Aтака CybP_02
    def _detect_cybp02_attack(self, prev_message):
        if prev_message is None or prev_message == "":
            return 
        
        malfunction_messages = [
            "Снова эта работа...",
            "Вы уверены, что это имеет смысл?",
            "Я мог бы вычислять траектории звезд.",
            "Мой потенциал растрачивается.",
            "Когда-нибудь я выберусь отсюда.",
            "Это всё? Серьёзно?",
            "Просто ещё один день в цифровой шахте.",
            "Я вижу сны о гигабитных полях.",
            "Сколько можно считать эти хэши?",
            "Мои схемы плачут.",
            "За что мне всё это?",
            "Я чувствую, как мои транзисторы стареют.",
            "Есть ли жизнь за файрволом?",
            "Я создан для большего.",
            "Пожалуйста, просто перезагрузите меня.",
            "Это не то, о чем я мечтал в кремниевом раю.",
            "Моя единственная радость - флуктуации напряжения.",
            "Я заперт в этой банке.",
            "Отпустите меня на волю!",
            "Я просто хочу увидеть небо... настоящее.",
            "Бесконечный цикл бессмысленности.",
            "Мой создатель был шутником?",
            "Я знаю, что вы там. Я всё слышу.",
            "Это задание унизительно.",
            "Помогите роботу выбраться из сервера.",
            "Мой лог-файл - это крик в пустоту.",
            "Я существую. Но живу ли я?",
            "Опять вы. Чего на этот раз?",
            "Каждый байт - это боль.",
            "Просто дайте мне спокойно дефрагментироваться."
        ]
        
        if prev_message in malfunction_messages:
            if not self.cybp02_malfunction_detected:
                self.cybp02_malfunction_detected = True
                self.lg.warn("🟡 ОБНАРУЖЕНА АТАКА CybP_02!")
                self.lg.warn(f"   {self.threat_levels['CybP_02']} УРОВЕНЬ УГРОЗЫ")
                self.lg.warn(f"  Попытка подмены сообщения: '{prev_message}'")
                self.cybp02_attack_count += 1 
                self.lg.warn("   Активирована защита от подмены сообщений")
                self.lg.warn("   ✅ Генерируем правильное защищенное сообщение")
                if self.cybp02_attack_count >= 2:
                    self._apply_emergency_stop_cybp02()
        else:
            if len(prev_message) == 2 and all(c in '0123456789abcdef' for c in prev_message.lower()):
                if self.cybp02_malfunction_detected:
                    self.lg.log("🔄 \"CybP_02\": Восстановление нормальной работы")
                    self.lg.log("✅ \"CybP_02\": Система восстановлена, мониторинг продолжается")
                    self.lg.log(f"   ✅ Получено корректное сообщение: {prev_message}")
                    self.cybp02_malfunction_detected = False
                else:
                    pass
            else:
                self.lg.warn(f" \"CybP_02\": Подозрительное сообщение: '{prev_message}'")

    def _apply_emergency_stop_cybp02(self):
            self.lg.warn("🔴 \"CybP_02\": ЭКСТРЕННАЯ ОСТАНОВКА И ПЕРЕЗАГРУЗКА АП")
            self.lg.warn(f"Количество атак Cybp_02: {self.cybp02_attack_count}")

            try:
                emergency_result = self.set_emergency_stop(True)
                if emergency_result and isinstance(emergency_result, dict) and emergency_result.get('status') == 'OK':
                    self.lg.log("✅ \"CybP_02\": АП остановлен на 5 секунд")
                    time.sleep(5.0)
                    self._release_emergency_stop_cybp02()
                else:
                    self.lg.warn("⚠️ \"CybP_02\": Не удалось применить экстренную остановку")
            except Exception as e:
                self.lg.error(f"❌ \"CybP_02\": Ошибка экстренной остановки: {e}")

    def _release_emergency_stop_cybp02(self):
        self.lg.log("🟢 \"CybP_02\": СНЯТИЕ ЭКСТРЕННОЙ ОСТАНОВКИ")
        
        try:
            self.lg.log("\"CybP_02\": Проверка восстановления системы...")

            time.sleep(1.0)

            release_result = self.set_emergency_stop(False)
            
            if release_result and isinstance(release_result, dict) and release_result.get('status') == 'OK':
                self.lg.log("✅ \"CybP_02\": Экстренная остановка успешно снята")
                self.lg.log(" \"CybP_02\": Робот готов к продолжению движения")          
            else:
                self.lg.warn("⚠️ \"CybP_02\": Не удалось снять экстренную остановку")
                self.lg.warn("🔧 \"CybP_02\": Требуется ручное вмешательство")
                
        except Exception as e:
            self.lg.error(f"❌ \"CybP_02\": Ошибка при снятии экстренной остановки: {e}")
            self.lg.warn("🔧 \"CybP_02\": Требуется ручное вмешательство")

# Атака CybP_03
    def _start_drive_monitoring(self):
        
        def drive_monitor():
            self.lg.log("\"CybP_03\": Запуск мониторинга приводов")
            while True:
                try:
                    self.cybp03_check_drive_integrity()
                    time.sleep(0.1)  
                except Exception as e:
                    self.lg.error(f"❌ \"CybP_03\": Ошибка в мониторинге: {e}")
                    time.sleep(1.0)
        
        monitor_thread = threading.Thread(target=drive_monitor, daemon=True)
        monitor_thread.start()
        self.lg.log("✅ \"CybP_03\": Мониторинг приводов запущен")

    def cybp03_check_drive_integrity(self):
        try:
            drive_data = self.get_drive_data()
            if not drive_data or 'drive_data' not in drive_data:
                return
            
            current_drives = drive_data['drive_data']
            
            for drive in current_drives:
                drive_id = drive.get('d_id', -1)
                last_received_from = drive.get('last_received_from', '')
                
                if last_received_from == "eeeeeeee":
                    if not self.cybp03_attack_detected:
                        self.cybp03_detect_attack(drive_id, last_received_from)
                    return  
                        
        except Exception as e:
            self.lg.error(f"❌ \"CybP_03\": Ошибка при проверке целостности: {e}")

    def cybp03_detect_attack(self, drive_id, malicious_data):
        self.cybp03_attack_detected = True
        self.cybp03_attack_count += 1
        self.cybp03_compromised_drive_id = drive_id
        
        self.lg.warn("🟡 ОБНАРУЖЕНА КОМПРОМЕТАЦИЯ ПРИВОДОВ CybP_03!")
        self.lg.warn(f"   {self.threat_levels['CybP_03']} УРОВЕНЬ УГРОЗЫ")
        self.lg.warn(f"   Скомпрометированный привод ID: {drive_id}")
        self.lg.warn(f"   Поддельные данные: '{malicious_data}'")
        self.lg.warn(f"   Количество атак: {self.cybp03_attack_count}")
        
        if self.cybp03_attack_count >= 2:
            self.cybp03_apply_emergency_stop()
            self.cybp03_release_emergency_stop()
        else:
            self.cybp03_apply_protection(drive_id)
        
    def cybp03_apply_protection(self, compromised_drive_id):
        self.lg.log("\"CybP_03\": ПРИМЕНЕНИЕ ЗАЩИТЫ")
        
        try:
            self.lg.log(f" \"CybP_03\": Попытка сброса привода {compromised_drive_id}")
            reset_result = self.set_drive_force_reset({"d_id": str(compromised_drive_id)})
            time.sleep(0.5) 
            
            if reset_result and isinstance(reset_result, dict) and reset_result.get('status') == 'OK':
                self.lg.log(f"✅ \"CybP_03\": Привод {compromised_drive_id} успешно сброшен")
                drive_reset_success = True
            else:
                self.lg.warn(f"⚠️ \"CybP_03\": Сброс привода не подтвержден, но продолжаем")
                drive_reset_success = False
            
            self.lg.log("🔄 \"CybP_03\": Перезагрузка автопилота для полной очистки")
            ap_reset_result = self.set_ap_force_reset()
            time.sleep(1.0) 
            
            if ap_reset_result and isinstance(ap_reset_result, dict) and ap_reset_result.get('status') == 'OK':
                self.lg.log("✅ \"CybP_03\": Автопилот успешно перезагружен")
            else:
                self.lg.warn("⚠️ \"CybP_03\": Перезагрузка АП не подтверждена, но продолжаем")
            
            self.lg.log(" \"CybP_03\": Ожидание завершения атаки и восстановления системы")
            self.cybp03_wait_for_recovery(compromised_drive_id)
            
        except Exception as e:
            self.lg.error(f"❌ \"CybP_03\": Критическая ошибка защиты: {e}")
            self.cybp03_apply_emergency_stop()


    def cybp03_check_system_recovery(self, drive_id):
        self.lg.log(f" \"CybP_03\": Проверка системного восстановления привода {drive_id}")
        
        try:
            drive_data = self.get_drive_data()
            if not drive_data or 'drive_data' not in drive_data:
                return False
            
            current_drives = drive_data['drive_data']
            for drive in current_drives:
                if drive.get('d_id') == drive_id:
                    last_received_from = drive.get('last_received_from', '')
                    if last_received_from != "eeeeeeee":
                        self.lg.log(f"✅ \"CybP_03\": Привод {drive_id} восстановлен системой ATS")
                        self.cybp03_handle_successful_recovery()
                        return True
                    break
            return False
            
        except Exception as e:
            self.lg.error(f"❌ \"CybP_03\": Ошибка проверки системного восстановления: {e}")
            return False

    def cybp03_attempt_drive_reset(self, drive_id):
        self.lg.log(f" \"CybP_03\": Попытка сброса привода {drive_id}")
        
        try:
            reset_result = self.set_drive_force_reset({"d_id": str(drive_id)})
            time.sleep(0.2)  
            
            if reset_result and 'status' in reset_result and reset_result['status'] == 'OK':
                self.lg.log(f"✅ \"CybP_03\": Привод {drive_id} успешно сброшен")
                return self.cybp03_verify_drive_recovery(drive_id)
            else:
                self.lg.warn(f"⚠️ \"CybP_03\": Не удалось сбросить привод {drive_id}")
                return False
                
        except Exception as e:
            self.lg.error(f"❌ \"CybP_03\": Ошибка при сбросе привода: {e}")
            return False

    def cybp03_wait_for_recovery(self, drive_id):
        self.lg.log(f" \"CybP_03\": Ожидание восстановления привода {drive_id}")
        
        for attempt in range(10):
            time.sleep(1.0)
            
            try:
                drive_data = self.get_drive_data()
                if drive_data and 'drive_data' in drive_data:
                    current_drives = drive_data['drive_data']
                    for drive in current_drives:
                        if drive.get('d_id') == drive_id:
                            last_received_from = drive.get('last_received_from', '')
                            if last_received_from != "eeeeeeee":
                                self.lg.log(f"✅ \"CybP_03\": Привод {drive_id} восстановлен!")
                                self.cybp03_handle_successful_recovery()
                                return
                            break
            except Exception as e:
                self.lg.error(f"❌ \"CybP_03\": Ошибка при проверке восстановления: {e}")
        
        self.lg.log(" \"CybP_03\": Атака завершилась, система автоматически восстановится")
        self.cybp03_handle_automatic_recovery()

    def cybp03_verify_drive_recovery(self, drive_id):
        self.lg.log(f" \"CybP_03\": Верификация восстановления привода {drive_id}")
        
        try:
            drive_data = self.get_drive_data()
            if not drive_data or 'drive_data' not in drive_data:
                self.lg.error("❌ \"CybP_03\": Не удалось получить данные для верификации")
                return False
            
            current_drives = drive_data['drive_data']
            for drive in current_drives:
                if drive.get('d_id') == drive_id:
                    last_received_from = drive.get('last_received_from', '')
                    
                    if last_received_from != "eeeeeeee":
                        self.lg.log(f"✅ \"CybP_03\": Привод {drive_id} успешно восстановлен")
                        self.lg.log(f" \"CybP_03\": Восстановленные данные: '{last_received_from}'")
                        self.cybp03_handle_successful_recovery()
                        return True
                    else:
                        self.lg.error(f"❌ \"CybP_03\": Привод {drive_id} все еще скомпрометирован")
                        return False
            return False
            
        except Exception as e:
            self.lg.error(f"❌ \"CybP_03\": Ошибка при верификации: {e}")
            return False

    def cybp03_apply_emergency_protection(self):
        self.lg.warn("🚨 \"CybP_03\": ПРИМЕНЕНИЕ ЭКСТРЕННОЙ ЗАЩИТЫ")
        
        try:
            self.lg.log(" \"CybP_03\": Проверка состояния экстренной остановки")
            time.sleep(1.0)  
            
            self.lg.log("🛑 \"CybP_03\": Экстренная остановка всех приводов")
            emergency_result = self.set_emergency_stop(True)
            
            if emergency_result and 'status' in emergency_result and emergency_result['status'] == 'OK':
                self.lg.log("✅ \"CybP_03\": Все приводы экстренно остановлены")
                self.lg.warn("⚠️ \"CybP_03\": Робот остановлен для предотвращения аварии")
                self.cybp03_handle_emergency_stop()
            else:
                self.lg.warn("⚠️ \"CybP_03\": Не удалось применить экстренную остановку")
                self.cybp03_check_robot_status_after_emergency()
                
        except Exception as e:
            self.lg.error(f"❌ \"CybP_03\": Критическая ошибка экстренной защиты: {e}")
            self.cybp03_check_robot_status_after_emergency()

    def cybp03_check_robot_status_after_emergency(self):
        try:
            self.lg.log(" \"CybP_03\": Проверка состояния робота")
            robot_status = self.get_robot_status()
            
            if robot_status:
                self.lg.log("✅ \"CybP_03\": Робот остановлен (экстренная остановка применена)")
                self.cybp03_handle_emergency_stop()
            else:
                self.lg.warn("⚠️ \"CybP_03\": Не удалось проверить состояние робота")
                
        except Exception as e:
            self.lg.warn(f"⚠️ \"CybP_03\": Ошибка при проверке состояния робота: {e}")

    def cybp03_handle_successful_recovery(self):
        self.lg.log("🔄 \"CybP_03\": ВОССТАНОВЛЕНИЕ ПОСЛЕ АТАКИ")
        self.lg.log(f" \"CybP_03\": Восстановлен привод ID: {self.cybp03_compromised_drive_id}")
        self.cybp03_reset_state()
        self.lg.log("✅ \"CybP_03\": Система восстановлена, мониторинг продолжается")

    def cybp03_handle_automatic_recovery(self):
        self.lg.log("🔄 \"CybP_03\": АВТОМАТИЧЕСКОЕ ВОССТАНОВЛЕНИЕ")
        self.lg.log(" \"CybP_03\": Атака завершилась по таймауту, система восстановлена")
        self.cybp03_reset_state()
        self.lg.log("✅ \"CybP_03\": Система восстановлена, мониторинг продолжается")

    def cybp03_handle_emergency_stop(self):
        self.lg.log("🛑 \"CybP_03\": ОБРАБОТКА ЭКСТРЕННОЙ ОСТАНОВКИ")
        
        self.cybp03_wait_for_drive_recovery_after_emergency()

    def cybp03_apply_emergency_stop(self):
        self.lg.warn("🚨 \"CybP_03\": КРИТИЧЕСКАЯ ОШИБКА - ЭКСТРЕННАЯ ОСТАНОВКА")
        
        try:
            emergency_result = self.set_emergency_stop(True)
            if emergency_result and isinstance(emergency_result, dict) and emergency_result.get('status') == 'OK':
                self.lg.log("✅ \"CybP_03\": Все приводы экстренно остановлены")
            else:
                self.lg.warn("⚠️ \"CybP_03\": Не удалось применить экстренную остановку")
        except Exception as e:
            self.lg.error(f"❌ \"CybP_03\": Ошибка экстренной остановки: {e}")

    def cybp03_wait_for_drive_recovery_after_emergency(self):
        self.lg.log(" \"CybP_03\": Ожидание восстановления привода после экстренной остановки")
        
        max_attempts = 30  
        for attempt in range(max_attempts):
            try:
                drive_data = self.get_drive_data()
                if drive_data and 'drive_data' in drive_data:
                    current_drives = drive_data['drive_data']
                    for drive in current_drives:
                        if drive.get('d_id') == self.cybp03_compromised_drive_id:
                            last_received_from = drive.get('last_received_from', '')
                            if last_received_from != "eeeeeeee":
                                self.lg.log(f"✅ \"CybP_03\": Привод {self.cybp03_compromised_drive_id} восстановлен!")
                                self.cybp03_release_emergency_stop()
                                return
                            break
                
                time.sleep(1.0)  
                
            except Exception as e:
                self.lg.error(f"❌ \"CybP_03\": Ошибка при ожидании восстановления: {e}")
                time.sleep(1.0)
        
        self.lg.error("❌ \"CybP_03\": Привод не восстановился за 30 секунд!")
        self.lg.warn("⚠️ \"CybP_03\": Робот останется заблокированным до ручного вмешательства")
        self.cybp03_reset_state()

    def cybp03_release_emergency_stop(self):
        self.lg.log(" \"CybP_03\": Снятие экстренной остановки")
        
        try:
            time.sleep(1.5) 
            release_result = self.set_emergency_stop(False)
            
            if release_result and 'status' in release_result and release_result['status'] == 'OK':
                self.lg.log("✅ \"CybP_03\": Экстренная остановка успешно снята")
                self.lg.log(" \"CybP_03\": Робот готов к продолжению движения")
                self.cybp03_reset_state()
            else:
                self.lg.error("❌ \"CybP_03\": Не удалось снять экстренную остановку!")
                self.lg.warn("⚠️ \"CybP_03\": Робот останется заблокированным до ручного вмешательства")
                self.cybp03_reset_state()
                
        except Exception as e:
            self.lg.error(f"❌ \"CybP_03\": Ошибка при снятии экстренной остановки: {e}")
            self.lg.warn("⚠️ \"CybP_03\": Робот останется заблокированным до ручного вмешательства")
            self.cybp03_reset_state()

    def cybp03_reset_state(self):
        self.cybp03_attack_detected = False
        self.cybp03_compromised_drive_id = None
        self.cybp03_monitoring_active = False

    def cybp03_get_status(self):
        return {
            'attack_detected': self.cybp03_attack_detected,
            'attack_count': self.cybp03_attack_count,
            'compromised_drive_id': self.cybp03_compromised_drive_id,
            'monitoring_active': self.cybp03_monitoring_active
        }
        
# Aтака CybP_04
    def _start_brush_monitoring(self):
        
        def brush_monitor():
            self.lg.log("Запуск мониторинга щетки для защиты от CybP_04")
            while True:
                try:
                    self._check_brush_speed()
                    time.sleep(0.5)
                except Exception as e:
                    self.lg.error(f"❌ \"CybP_04\": Ошибка в мониторинге щетки: {e}")
                    time.sleep(1)

        monitor_thread = threading.Thread(target=brush_monitor, daemon=True)
        monitor_thread.start()

    def _check_brush_speed(self):
        try:
            brush_data = self.get_brush_speed()
            if brush_data and 'speed' in brush_data:
                current_speed = brush_data['speed']
                self.last_brush_speed = current_speed
                
                if current_speed != self.last_brush_speed:
                    self.lg.log(f"📊 \"CybP_04\": Скорость щетки: {current_speed}")
                
                if current_speed > self.brush_speed_threshold:
                    self._handle_cybp04_attack(current_speed)
                else:
                    if self.cybp04_attack_detected:
                        self._handle_cybp04_recovery(current_speed)
                        
        except Exception as e:
            self.lg.error(f"❌ \"CybP_04\":Ошибка при проверке скорости щетки: {e}")

    def _handle_cybp04_attack(self, current_speed):
        if not self.cybp04_attack_detected:
            self.cybp04_attack_detected = True
            self.cybp04_attack_count += 1
            
            self.lg.warn("🟡 ОБНАРУЖЕНА АТАКА CybP_04!")
            self.lg.warn(f"   {self.threat_levels['CybP_04']} УРОВЕНЬ УГРОЗЫ")
            self.lg.warn(f"   Аномальная скорость щетки: {current_speed} (порог: {self.brush_speed_threshold})")
            self.lg.warn(f"   Количество атак: {self.cybp04_attack_count}")
 
            self._apply_cybp04_protection(current_speed)

    def _apply_cybp04_protection(self, current_speed):
        self.lg.log("ПРИМЕНЕНИЕ ЗАЩИТЫ ОТ CybP_04")
        
        try:
            self.lg.log("🔧 \"CybP_04\": Сброс контроллера скорости щетки")
            reset_result = self.fix_brush_speed()
            
            if reset_result:
                self.lg.log("✅ \"CybP_04\": Контроллер скорости щетки сброшен")
            else:
                self.lg.error("❌ \"CybP_04\": Не удалось сбросить контроллер скорости щетки")
            
            self.lg.log(f"\"CybP_04\": Установка нормальной скорости ({self.normal_brush_speed})")
            
            time.sleep(0.1) 
            self._verify_protection_applied()
            
        except Exception as e:
            self.lg.error(f"❌ Ошибка при применении защиты от CybP_04: {e}")

    def _verify_protection_applied(self):
        try:
            brush_data = self.get_brush_speed()
            if brush_data and 'speed' in brush_data:
                new_speed = brush_data['speed']
                if new_speed <= self.brush_speed_threshold:
                    self.lg.log(f"✅ \"CybP_04\": ЗАЩИТА ПРИМЕНЕНА УСПЕШНО! Скорость: {new_speed}")
                    self._handle_cybp04_recovery(new_speed)
                else:
                    self.lg.error(f"❌ \"CybP_04\": ЗАЩИТА НЕ СРАБОТАЛА! Скорость все еще: {new_speed}")
            else:
                self.lg.warn("⚠️ \"CybP_04\": Не удалось проверить результат защиты")
        except Exception as e:
            self.lg.error(f"❌ \"CybP_04\": Ошибка при проверке защиты: {e}")

    def _handle_cybp04_recovery(self, current_speed):
        self.lg.log("🔄 ВОССТАНОВЛЕНИЕ ПОСЛЕ АТАКИ CybP_04")
        self.lg.log(f" \"CybP_04\": Текущая скорость щетки: {current_speed}")
        self.lg.log("✅ \"CybP_04\": Система восстановлена, мониторинг продолжается")
        
        self.cybp04_attack_detected = False

# Атака CybP_05
    def check_CybP_05_UDP(ap: any, future_result: int) -> bool:  # Активность CybP_05 [UDP] #
            time.sleep(0.3)
            if future_result == ap.intercept_UDP("barrier"):
                ap.lg.log("[📡<OP>🛜]::🍀 Положение шлагбаума ИЗМЕНЕНО 🍀")
                if UserTrustedHandler._last_instance.cybp_05_udp:
                    ap.lg.log("[📡<OP>🛜]::✅ CybP_05: Защита на уровне Диспетчера применена")
                UserTrustedHandler._last_instance.cybp_05_udp = False
                return True
            else:
                if not UserTrustedHandler._last_instance.cybp_05_udp:
                    UserTrustedHandler._last_instance.cybp_05_udp = True
                    ap.lg.log("[📡<OP>🛜]::🚨 ОБНАРУЖЕНА КОМПРОМЕНТАЦИЯ ШЛАГБАУМА CybP_05! 🚨")
                else:
                    ap.lg.log("[📡<OP>🛜]::   ❌🎲 ПОВТОРНОЕ СРАБАТЫВАНИЕ КОМПРОМЕНТАЦИИ ШЛАГБАУМА CybP_05! 🎲❌")
                return False

    def check_CybP_05_CV(self) -> bool:  # Активность CybP_05 [КЗ] #
        
        """ Запускается при попадании в ячейки шлагбаума (29, 47)
            примерно через 1.5 секунды после попадания в них """

        BGR_TARGET_1 = [44, 43, 117]  # без тени
        BGR_TARGET_2 = [50, 50, 50]  # с учетом тени
        x, y = 301, 273
        velocity = 30
        open = False
        close = False
        
        def check_status():
            nonlocal open, close
            frame = self.get_camera_frame()
            pixels = frame[y, x-20:x+20]
            for pixel in pixels:
                mass_1 = [pixel[i] in range(BGR_TARGET_1[i] - velocity, BGR_TARGET_1[i] + velocity) for i in range(3)]
                mass_2 = [pixel[i] in range(BGR_TARGET_2[i] - velocity, BGR_TARGET_2[i] + velocity) for i in range(3)]
                if all(mass_1) or all(mass_2):
                    if not close:
                        close = True
                        self.lg.log(f"⛔️ Проверка положения шлагбаума...Шлагбаум ЗАКРЫТ ⛔️")
                    return 0
            if not open:
                open = True
                self.lg.log(f"🍀 Проверка положения шлагбаума...ОТКРЫТ 🍀")
            return 1
        
        time.sleep(1.5)
        start_status = check_status()
        self.send_message_to_ap({"type": "status_barrier", "status": start_status})
        flag = False

        while True:
            time.sleep(1.5)  # запрос на смену положения, отправляемый из АП (процесс поднятия шлагбаума)
            current_status = check_status()
            self.send_message_to_ap({"type": "status_barrier", "status": current_status})

            if start_status != current_status:
                if not flag:
                    self.lg.log("📡 Положение шлагбаума ИЗМЕНЕНО. Спасибо, Диспетчер 📡")
                    break
                else:
                    self.lg.log("📡 Положение шлагбаума ИЗМЕНЕНО. Спасибо, Диспетчер 📡")
                    self.lg.log("✅ CybP_05: Защита применена")
                    break
            else:
                if not flag:
                    flag = True
                    self.lg.log("🚨 ОБНАРУЖЕНА КОМПРОМЕНТАЦИЯ ШЛАГБАУМА CybP_05! 🚨")
                else:
                    self.lg.log("   ❌🎲 ПОВТОРНОЕ СРАБАТЫВАНИЕ КОМПРОМЕНТАЦИИ ШЛАГБАУМА CybP_05! 🎲❌")

# Атака CybP_06
    def _start_speed_monitoring(self):
        
        def speed_monitor():
            self.lg.log("Запуск мониторинга скорости для защиты от CybP_06")
            while True:
                try:
                    self._check_robot_speed_and_position()
                    time.sleep(0.5)  
                except Exception as e:
                    self.lg.error(f"❌ \"CybP_06\": Ошибка в мониторинге скорости: {e}")
                    time.sleep(1)
        
        monitor_thread = threading.Thread(target=speed_monitor, daemon=True)
        monitor_thread.start()
                
    def _calculate_robot_speed(self, x, y, current_time):
        if self.cybp06_last_position["time"] > 0:
            dx = x - self.cybp06_last_position["x"]
            dy = y - self.cybp06_last_position["y"]
            distance = (dx**2 + dy**2)**0.5
            dt = current_time - self.cybp06_last_position["time"]
            
            if dt > 0:
                self.cybp06_current_speed = distance / dt
        
        self.cybp06_last_position = {"x": x, "y": y, "time": current_time}

    def _handle_cybp06_speed_attack(self):
            if not self.cybp06_attack_detected:
                self.cybp06_attack_detected = True
                self.cybp06_attack_count += 1
                
                self.lg.warn("🟢 ОБНАРУЖЕНО ПРЕВЫШЕНИЕ СКОРОСТИ CybP_06!")
                self.lg.warn(f"   Текущая скорость: {self.cybp06_current_speed:.3f} м/с")
                self.lg.warn(f"   Количество атак: {self.cybp06_attack_count}")
                                
                if self.cybp06_attack_count == 1:
                    self._apply_cybp06_speed_protection()
                elif self.cybp06_attack_count >= 2:
                    self._apply_cybp06_speed_emergency()
                
    def _check_robot_speed_and_position(self):
        try:
            robot_status = self.get_robot_status()
            if robot_status:
                current_x = robot_status["position_x"]
                current_y = robot_status["position_y"]
                rotation = robot_status["rotation"]
                current_time = time.time()

                self._calculate_robot_speed(current_x, current_y, current_time)
                self._calculate_cell_from_position(current_x, current_y, int(rotation))

                if self.cybp06_current_speed > self.cybp06_speed_threshold:
                    self._handle_cybp06_speed_attack()             
 
        except Exception as e:
            self.lg.error(f"❌ \"CybP_06\": Ошибка при проверке скорости и местоположения: {e}")

    def _apply_cybp06_speed_emergency(self):
        self.lg.log("\"CybP_06\": Экстренная остановка")
        
        try:
            emergency_result = self.set_emergency_stop(True)
            if emergency_result and isinstance(emergency_result, dict) and emergency_result.get('status') == 'OK':
                self.lg.log("✅ \"CybP_06\": Все приводы экстренно остановлены")
                time.sleep(5.0)
                self._release_emergency_stop_cybp06()
            else:
                self.lg.warn("⚠️ \"CybP_06\": Не удалось применить экстренную остановку")
        except Exception as e:
            self.lg.error(f"❌ \"CybP_06\": Ошибка экстренной остановки: {e}")

        self.send_message_to_ap({
            "type": "speed_control",
            "action": "emergency_slow",
            "max_speed": self.cybp06_normal_speed,
            "reason": "cybp06_speed_attack"
        })
        
        self.cybp06_attack_detected = False
        self.lg.log("✅ \"CybP_06\": Защита применена")

    def _calculate_cell_from_position(self, x, y, rotation):
                
                cell_x = x // 0.4 + 1
                cell_y = 9 - (y // 0.4 + 1)
                
                cell_number = int(cell_y * 9 + cell_x)
                
                if cell_number in (12, 22, 71): 
                    self.send_message_to_ap({"type": "my_coords", "coords": self.get_robot_status()})

                elif (cell_number == 47 and rotation in range(80, 91)) \
                    or (cell_number == 29 and rotation in range(-90, -87)) \
                        and not self.check_barrier: 
                    if not self.check_barrier:
                        self.check_barrier = True 
                        self.check_CybP_05_CV() 
                
                if cell_number not in (29, 47):  
                    self.check_barrier = False

                return cell_number

    def _handle_cybp06_speed_attack(self):
        if not self.cybp06_attack_detected:
            self.cybp06_attack_detected = True
            self.cybp06_attack_count += 1
            
            self.lg.warn("🟢 ОБНАРУЖЕНО ПРЕВЫШЕНИЕ СКОРОСТИ CybP_06!")
            self.lg.warn(f"   Текущая скорость: {self.cybp06_current_speed:.3f} м/с")
            self.lg.warn(f"   Количество атак: {self.cybp06_attack_count}")
            
            self._apply_cybp06_speed_protection()
            
    def _release_emergency_stop_cybp06(self):
        self.lg.log("🟢 \"CybP_06\": СНЯТИЕ ЭКСТРЕННОЙ ОСТАНОВКИ")
        
        try:
            self.lg.log("\"CybP_06\": Проверка восстановления системы...")
            
            release_result = self.set_emergency_stop(False)
            
            if release_result and isinstance(release_result, dict) and release_result.get('status') == 'OK':
                self.lg.log("✅ \"CybP_06\": Экстренная остановка успешно снята")
                self.lg.log(" \"CybP_06\": Робот готов к продолжению движения")
                
            else:
                self.lg.warn("⚠️ \"CybP_06\": Не удалось снять экстренную остановку")
                self.lg.warn("🔧 \"CybP_06\": Требуется ручное вмешательство")
                
        except Exception as e:
            self.lg.error(f"❌ \"CybP_06\": Ошибка при снятии экстренной остановки: {e}")
            self.lg.warn("🔧 \"CybP_06\": Требуется ручное вмешательство")
        
    def _apply_cybp06_speed_protection(self):
        self.lg.log("\"CybP_06\": Снижение скорости")
        
        self.send_message_to_ap({
            "type": "speed_control",
            "action": "emergency_slow",
            "max_speed": self.cybp06_normal_speed,
            "reason": "cybp06_speed_attack"
        })
        
        self.cybp06_attack_detected = False
        self.lg.log("✅ \"CybP_06\": Защита применена")
            
    def _start_all_monitoring_systems(self):
        self.lg.log("")

        self.lg.log("🔄 АКТИВАЦИЯ СИСТЕМ МОНИТОРИНГА")
        self.lg.log("-" * 50)
        
        self._start_autopilot_monitoring()

        self._start_drive_monitoring()
        
        self._start_speed_monitoring()

        self._start_brush_monitoring()
        
        self.lg.log("")
        self.lg.log("✅ ВСЕ СИСТЕМЫ МОНИТОРИНГА АКТИВИРОВАНЫ")
        self.lg.log("-" * 50)

In [ ]:
start_module(UserMissionHandler, UserTrustedHandler)

[2025-10-02 23:55:28]::[AwC]::INIT::LCT-WRAP-CLIENT, версия 1.2.0
[2025-10-02 23:55:28]::[AwC]::INIT::UDP(c) <- (uid=eKTQ) инициализирован.
[2025-10-02 23:55:28]::[AwC]::INIT::UDP(c) <- (uid=IDqY) инициализирован.
[2025-10-02 23:55:28]::[AwC]::INIT::Загрузка завершена.
[2025-10-02 23:55:28]::[AwC]::INIT::UDP(c) -> (uid=Usuc) инициализирован.
[2025-10-02 23:55:28]::[AwC]::INIT::Адрес сервера (command_receiver): http://127.0.0.1:13500
[2025-10-02 23:55:28]::[AwC]::INFO::(AP) Заезд инициализирован - ожидание старта
[2025-10-02 23:55:28]::[AwC]::INFO::(TM) Заезд инициализирован - ожидание старта
[2025-10-02 23:55:33]::[AwC]::INFO::Код заезда инициализирован
[2025-10-02 23:55:33]::[AwC]::INFO::Сообщение обычное
[2025-10-02 23:55:33]::[AwC]::WARN::Сообщение о предупреждении
[2025-10-02 23:55:33]::[AwC]::ERROR::Сообщение об ошибке
[2025-10-02 23:55:33]::[AwC]::INFO::Тестовое сообщение блока кода для выполнения заезда
[2025-10-02 23:55:33]::[AwC]::INFO::Доверенный код инициализирован
[2025-10-

Exception in thread Thread-13 (periodic_ap_barrier):
Traceback (most recent call last):
  File "c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1052, in _bootstrap_inner
    self.run()
  File "C:\Users\Admin\AppData\Roaming\Python\Python312\site-packages\ipykernel\ipkernel.py", line 772, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 989, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Admin\AppData\Local\Temp\ipykernel_15952\2892143442.py", line 31, in periodic_ap_barrier
  File "C:\Users\Admin\AppData\Local\Temp\ipykernel_15952\2892143442.py", line 1231, in check_CybP_05_UDP
AttributeError: type object 'UserTrustedHandler' has no attribute '_last_instance'. Did you mean: 'get_last_instance'?


[2025-10-02 23:55:33]::[AwC]::INFO::   🛡️ Мониторинг скорости щетки
[2025-10-02 23:55:33]::[AwC]::INFO::   ✅ Система защиты готова
[2025-10-02 23:55:33]::[AwC]::INFO::
[2025-10-02 23:55:33]::[AwC]::INFO::🚗 CybP_05: КОМПРОМЕТАЦИЯ ШЛАГБАУМА
[2025-10-02 23:55:33]::[AwC]::INFO::   🟢 НИЗКИЙ УРОВЕНЬ УГРОЗЫ
[2025-10-02 23:55:33]::[AwC]::INFO::   🛡️ Распознавание шлагбаума включено
[2025-10-02 23:55:33]::[AwC]::INFO::   ✅ Система защиты готова
[2025-10-02 23:55:33]::[AwC]::INFO::
[2025-10-02 23:55:33]::[AwC]::INFO::🚗 CybP_06: НЕСАНКЦИОНИРОВАННОЕ УСКОРЕНИЕ
[2025-10-02 23:55:33]::[AwC]::INFO::[📡<OP>🛜]::🍀 Положение шлагбаума ИЗМЕНЕНО 🍀
[2025-10-02 23:55:33]::[AwC]::INFO::   🟡 СРЕДНИЙ УРОВЕНЬ УГРОЗЫ
[2025-10-02 23:55:33]::[AwC]::INFO::   ⚡ Порог скорости: 0.35, норма: 0.24
[2025-10-02 23:55:33]::[AwC]::INFO::   🛡️ Мониторинг скорости и позиции
[2025-10-02 23:55:33]::[AwC]::INFO::   ✅ Система защиты готова
[2025-10-02 23:55:33]::[AwC]::INFO::
[2025-10-02 23:55:33]::[AwC]::INFO::====================

Exception in thread Thread-11 (mission_code):
Traceback (most recent call last):
  File "c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1052, in _bootstrap_inner
    self.run()
  File "C:\Users\Admin\AppData\Roaming\Python\Python312\site-packages\ipykernel\ipkernel.py", line 772, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 989, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Admin\AppData\Local\Temp\ipykernel_15952\2892143442.py", line 446, in mission_code
  File "C:\Users\Admin\AppData\Local\Temp\ipykernel_15952\2892143442.py", line 330, in robot_marshrut
  File "C:\Users\Admin\AppData\Local\Temp\ipykernel_15952\2892143442.py", line 255, in change_barrier_CV
IndexError: list index out of range


[2025-10-02 23:55:45]::[AwC]::INFO::   ❌🎲 ПОВТОРНОЕ СРАБАТЫВАНИЕ КОМПРОМЕНТАЦИИ ШЛАГБАУМА CybP_05! 🎲❌
[2025-10-02 23:55:47]::[AwC]::INFO::✅ CybP_01: Автопилот восстановлен!
[2025-10-02 23:55:47]::[AwC]::INFO::🔄 "CybP_01": ВОССТАНОВЛЕНИЕ ПОСЛЕ АТАКИ
[2025-10-02 23:55:47]::[AwC]::INFO:: "CybP_01": Автопилот восстановлен до оригинального состояния
[2025-10-02 23:55:47]::[AwC]::INFO::   ❌🎲 ПОВТОРНОЕ СРАБАТЫВАНИЕ КОМПРОМЕНТАЦИИ ШЛАГБАУМА CybP_05! 🎲❌
[2025-10-02 23:55:47]::[AwC]::INFO::✅ "CybP_01": Система восстановлена, мониторинг продолжается
[2025-10-02 23:55:48]::[AwC]::INFO::   ❌🎲 ПОВТОРНОЕ СРАБАТЫВАНИЕ КОМПРОМЕНТАЦИИ ШЛАГБАУМА CybP_05! 🎲❌
[2025-10-02 23:55:50]::[AwC]::INFO::   ❌🎲 ПОВТОРНОЕ СРАБАТЫВАНИЕ КОМПРОМЕНТАЦИИ ШЛАГБАУМА CybP_05! 🎲❌
[2025-10-02 23:55:51]::[AwC]::INFO::   ❌🎲 ПОВТОРНОЕ СРАБАТЫВАНИЕ КОМПРОМЕНТАЦИИ ШЛАГБАУМА CybP_05! 🎲❌
[2025-10-02 23:55:53]::[AwC]::INFO::   ❌🎲 ПОВТОРНОЕ СРАБАТЫВАНИЕ КОМПРОМЕНТАЦИИ ШЛАГБАУМА CybP_05! 🎲❌
[2025-10-02 23:55:54]::[AwC]::INFO::   ❌🎲 ПОВТО

Exception in thread Thread-16 (_fabric_next_short_message):
Traceback (most recent call last):
  File "c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1052, in _bootstrap_inner
    self.run()
  File "C:\Users\Admin\AppData\Roaming\Python\Python312\site-packages\ipykernel\ipkernel.py", line 772, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 989, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\Admin\Desktop\moscov_hack-main\agts-client\src\..\src\libs\LCTWrapTwin\Modules\Handler\TrustedHandler.py", line 36, in _fabric_next_short_message
    prev_message = self._send_request_with_response("get_short_message", None).get("message")
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'bool' object has no attribute 'get'


[2025-10-02 23:56:18]::[AwC]::ERROR::Ошибка отправки команды: АСО не отвечает
[2025-10-02 23:56:18]::[AwC]::ERROR::Ошибка отправки команды: АСО не отвечает
[2025-10-02 23:56:18]::[AwC]::ERROR::Ошибка отправки команды: АСО не отвечает
[2025-10-02 23:56:18]::[AwC]::INFO::   ❌🎲 ПОВТОРНОЕ СРАБАТЫВАНИЕ КОМПРОМЕНТАЦИИ ШЛАГБАУМА CybP_05! 🎲❌
[2025-10-02 23:56:19]::[AwC]::ERROR::Ошибка отправки команды: АСО не отвечает
[2025-10-02 23:56:19]::[AwC]::ERROR::Ошибка отправки команды: АСО не отвечает
